In [104]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [105]:
#setting up selenium web driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm")

In [106]:
import time 
from bs4 import BeautifulSoup
import re

In [107]:
import time
import re
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Empty list to store job data
job_data = []


# Define a function to scrape jobs from the current page
def scrape_page():
    # Get the page source and parse it
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find all job cards on the page
    job_cards = soup.find_all('li', class_=lambda x: x and 'JobsList_jobListItem' in x)
    
    for job in job_cards:
        try:
            title_tag = job.select_one('a[data-test="job-title"]')
            title = title_tag.get_text(strip=True) if title_tag else "N/A"
        except Exception as e:
            title = "N/A"
            print(f"Error extracting title: {e}")
        
        try:
            company_raw = job.find('div', class_='EmployerProfile_profileContainer__63w3R').text.strip()
            company = re.sub(r'\d\.\d$', '', company_raw)  # Remove trailing rating like "4.8"
        except Exception as e:
            company = "N/A"
            print(f"Error extracting company: {e}")
        
        try:
            location = job.find('div', {'data-test': 'emp-location'}).text.strip()
        except Exception as e:
            location = "N/A"
            print(f"Error extracting location: {e}")
        
        try:
            salary = job.find('div', {'data-test': 'detailSalary'}).text.strip()
        except Exception as e:
            salary = "N/A"
            print(f"Error extracting salary: {e}")

        job_data.append([title, company, location, salary])

# Function to click "Show more jobs" until all jobs are loaded
def load_all_jobs(max_clicks=15):
    click_count = 0
    while click_count < max_clicks:
        try:
            # Wait for the "Show more jobs" button to be clickable
            next_button = WebDriverWait(driver, 10).until(
                 EC.element_to_be_clickable((By.XPATH, "//button[.//span[contains(text(), 'Show more jobs')]]"))
            )
            
            # If the button is found, click it
            next_button.click()
            click_count += 1  # Increment the click counter
            print(f"Clicked 'Show more jobs' {click_count} times.")
            
            time.sleep(5)  # Wait for new jobs to load
            
            # Scrape the newly loaded jobs
            scrape_page()
            
        except Exception as e:
            print(f"Error clicking the 'Show more jobs' button: {e}")
            break  # Break the loop if there's an issue (e.g., button not found)


# Wait for page to load
time.sleep(5)

# Start loading and scraping all jobs
scrape_page()  # Scrape the initial set of jobs
load_all_jobs(max_clicks=15)  # Load more jobs by clicking the "Show more jobs" button up to 15 times

# Create the DataFrame with the data
df = pd.DataFrame(job_data, columns=['Job Title', 'Company', 'Location', 'Salary'])

# Print the DataFrame
print(df)



Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Clicked 'Show more jobs' 1 times.
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Clicked 'Show more jobs' 2 times.
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute 'text'
Error extracting salary: 'NoneType' object has no attribute '

In [108]:
print(df.head(110))

                                     Job Title                  Company  \
0                     Junior Software Engineer  CommIT Enterprises, Inc   
1           Senior Software Engineer, Frontend               Postscript   
2                 Software Engineer, Referrals                    Chime   
3    Software Engineer, Cybersecurity - Remote                    Optum   
4                          Software Engineer 1               US Venture   
..                                         ...                      ...   
105                 Mobius Software Engineer I     Autonomous Solutions   
106                          Software Engineer  Amplify Education, Inc.   
107              IT Software Engineer - Remote       Prime Therapeutics   
108              Entry Level Software Engineer                  Amentum   
109                        Software Engineer 5                  Netflix   

               Location                         Salary  
0          Quantico, VA   $61K - $94K (Gla

In [97]:
print(df)

                              Job Title                             Company  \
0              Junior Software Engineer             CommIT Enterprises, Inc   
1    Senior Software Engineer, Frontend                          Postscript   
2          Software Engineer, Referrals                               Chime   
3                   Software Engineer 1                          US Venture   
4        Sr. Software Engineer, Backend                            Poshmark   
..                                  ...                                 ...   
205                   Software Engineer                             TPS LLC   
206                   Software Engineer                            ADS, Inc   
207                   Software Engineer  Vytal Filtration Technologies Inc.   
208                 Software Engineer I                             TopBloc   
209                   Software Engineer                               Pryon   

              Location                         Sala

In [110]:
rows, cols = df.shape
print(f"Number of rows: {rows}")
print(f"Number of columns: {cols}")

Number of rows: 4080
Number of columns: 4


In [111]:
df.to_csv('D:/glassdoor_jobs.csv', index=False)